In [1]:
#!pip install --upgrade deepface

In [ ]:
conda create -n deepface_env python=3.9

Channels:
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3/envs/deepface_env

  added / updated specs:
    - python=3.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.9.24  |       hca03da5_0         131 KB
    openssl-3.0.15             |       h80987f9_0         4.3 MB
    pip-24.2                   |   py39hca03da5_0         2.2 MB
    python-3.9.20              |       hb885b13_1        12.5 MB
    setuptools-75.1.0          |   py39hca03da5_0         1.6 MB
    tzdata-2024b               |       h04d1e81_0         115 KB
    wheel-0.44.0               |   py39hca03da5_0         110 KB
    ------------------------------------------------------------
                                           Total:        21.1 MB

The following NEW packages will be INSTALLED:

  ca-certificates    pkgs/main/o

In [ ]:
[y]

In [27]:
!pip install --upgrade tensorflow keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0


In [31]:
!pip install --upgrade deepface

  Using cached deepface-0.0.93-py3-none-any.whl.metadata (30 kB)
Using cached deepface-0.0.93-py3-none-any.whl (108 kB)
  Attempting uninstall: deepface
    Found existing installation: deepface 0.0.72
    Uninstalling deepface-0.0.72:
      Successfully uninstalled deepface-0.0.72


In [33]:
from deepface import DeepFace
import cv2

# Load a sample image
img = cv2.imread("/Users/muttakibismoy/Downloads/Bismoy_1.jpg")  # Replace with your image path

# Test DeepFace emotion analysis on a single image with specific backend
result = DeepFace.analyze(img, actions=['emotion'], detector_backend='mtcnn')
print(result)

ImportError: cannot import name '__version__' from 'deepface' (/opt/anaconda3/lib/python3.12/site-packages/deepface/__init__.py)

In [ ]:
#!pip install tensorflow keras opencv-python-headless

In [ ]:
#!pip install deepface==0.0.72

In [ ]:
import cv2
from deepface import DeepFace

# Load the image
img = cv2.imread("/Users/muttakibismoy/Downloads/2.jpg")

# Use OpenCV to detect faces (Haar cascades as an example)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.1, 4)

# If a face is found, crop and analyze it
for (x, y, w, h) in faces:
    face = img[y:y+h, x:x+w]  # Crop the face
    
    # Analyze the cropped face for emotions
    result = DeepFace.analyze(face, actions=['emotion'], detector_backend='mtcnn')
    print(result)

In [ ]:
#!pip install deepface opencv-python matplotlib

In [13]:
import cv2
import matplotlib.pyplot as plt
from deepface import DeepFace
import numpy as np

ImportError: cannot import name 'LocallyConnected2D' from 'tensorflow.keras.layers' (/opt/anaconda3/lib/python3.12/site-packages/keras/_tf_keras/keras/layers/__init__.py)

In [ ]:
def analyze_video_emotions(video_path, frame_interval=30, detector_backend='dlib'):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Get video frame rate to convert frames to seconds
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    frame_count = 0
    time_stamps = []  # Store timestamps in seconds
    emotions_over_time = []  # Store detected emotions per frame
    
    # Loop through the video frames
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Exit if no more frames
        
        # Analyze every 'frame_interval' frame (to save processing time)
        if frame_count % frame_interval == 0:
            print(f"Processing frame {frame_count}...")  # Debug print
            time_in_seconds = frame_count / fps  # Convert frame count to seconds
            time_stamps.append(time_in_seconds)
            
            try:
                # Step 4: Perform emotion detection on each frame
                result = DeepFace.analyze(frame, actions=['emotion'], detector_backend=detector_backend, enforce_detection=False)

                # Ensure we have a valid result before attempting to access it
                if 'emotion' in result:
                    emotions = result['emotion']
                    emotions_over_time.append(emotions)  # Append detected emotions
                    print(f"Detected emotions in frame {frame_count}: {emotions}")
                else:
                    print(f"No face detected in frame {frame_count}")
                    emotions_over_time.append(None)  # Append None if no face detected

                # For debugging purposes, show the frame with the detected face
                if frame_count == 0 or frame_count == 150:
                    cv2.imshow(f"Frame {frame_count}", frame)
                    cv2.waitKey(0)  # Wait for key press to close the image window
                    cv2.destroyAllWindows()

            except Exception as e:
                print(f"Error analyzing frame {frame_count}: {e}")
                emotions_over_time.append(None)  # Append None in case of errors
        
        frame_count += 1
    
    # Release the video capture object
    cap.release()
    
    return time_stamps, emotions_over_time

In [ ]:
def plot_emotions_over_time(time_stamps, emotions_over_time):
    if len(emotions_over_time) == 0:
        print("No emotion data to plot.")
        return

    # Initialize data containers for the different emotions
    emotion_keys = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    emotion_data = {key: [] for key in emotion_keys}  # To store emotion intensities

    # Organize emotion intensities by time
    for emotions in emotions_over_time:
        if emotions is not None:  # If emotions were detected in this frame
            for key in emotion_keys:
                emotion_data[key].append(emotions.get(key, 0))  # Append emotion or 0 if missing
        else:
            # Append 0 for each emotion if no emotions were detected
            for key in emotion_keys:
                emotion_data[key].append(0)

    # Convert time_stamps to a numpy array for easier plotting
    time_stamps = np.array(time_stamps)
    
    # Step 6: Plot the emotion data over time
    plt.figure(figsize=(12, 6))
    
    for emotion, values in emotion_data.items():
        plt.plot(time_stamps, values, label=emotion)
    
    plt.title("Emotion Changes Over Time")
    plt.xlabel("Time (seconds)")
    plt.ylabel("Emotion Intensity")
    plt.legend(loc="upper right")
    plt.grid(True)
    plt.show()

In [ ]:
def main(video_path):
    time_stamps, emotions_over_time = analyze_video_emotions(video_path, frame_interval=30, detector_backend='mtcnn')
    if len(emotions_over_time) > 0:
        plot_emotions_over_time(time_stamps, emotions_over_time)
    else:
        print("No emotions were detected in the video.")

In [ ]:
video_path = "/Users/muttakibismoy/Downloads/archive/HACER/sitdown_angry_1_h.mp4"
main(video_path)